# Enron Case-Person of Interest

In [1]:
#!/usr/bin/python
## Author is Rajdeep
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
##  tester.py is modified becuase of renaming and deprecation of cross_validation sub-module to model_selection
from tester import dump_classifier_and_data
##  tester.py is modified becuase of renaming and deprecation of cross_validation sub-module to model_selection

In [2]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list=['poi','salary']

In [3]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [4]:
data_dict

{'ALLEN PHILLIP K': {'bonus': 4175000,
  'deferral_payments': 2869717,
  'deferred_income': -3081055,
  'director_fees': 'NaN',
  'email_address': 'phillip.allen@enron.com',
  'exercised_stock_options': 1729541,
  'expenses': 13868,
  'from_messages': 2195,
  'from_poi_to_this_person': 47,
  'from_this_person_to_poi': 65,
  'loan_advances': 'NaN',
  'long_term_incentive': 304805,
  'other': 152,
  'poi': False,
  'restricted_stock': 126027,
  'restricted_stock_deferred': -126027,
  'salary': 201955,
  'shared_receipt_with_poi': 1407,
  'to_messages': 2902,
  'total_payments': 4484442,
  'total_stock_value': 1729541},
 'BADUM JAMES P': {'bonus': 'NaN',
  'deferral_payments': 178980,
  'deferred_income': 'NaN',
  'director_fees': 'NaN',
  'email_address': 'NaN',
  'exercised_stock_options': 257817,
  'expenses': 3486,
  'from_messages': 'NaN',
  'from_poi_to_this_person': 'NaN',
  'from_this_person_to_poi': 'NaN',
  'loan_advances': 'NaN',
  'long_term_incentive': 'NaN',
  'other': 'NaN'

In [5]:
### Data exploration
### Task 2: Remove outliers
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

In [6]:
#Number of suspects
name_data_point=data_dict.keys()

In [7]:
print 'Total number of datapoints are {}'.format(len(name_data_point))

Total number of datapoints are 146


In [8]:
#Number of features
total_features=len (data_dict[name_data_point[0]])
print 'Total number of features are {}'.format(total_features)


Total number of features are 21


In [9]:
features_temp=data_dict[name_data_point[0]].keys()

In [10]:
#For convinience of data cleaning (Removing NaN) using pandas (Ref: Data visulization)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
unwanted_features=["poi","email_address"]

features_temp=[ele for ele in features_temp if ele not in unwanted_features]
# to make the first element in the list "poi"
features_temp=["poi"]+features_temp
feature_data= featureFormat(data_dict,features_temp,remove_NaN=False)


In [12]:
temp_df=pd.DataFrame(data=feature_data,columns=features_temp, index=name_data_point)
print "With NaN"
print temp_df.info()

With NaN
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 20 columns):
poi                          146 non-null float64
salary                       95 non-null float64
to_messages                  86 non-null float64
deferral_payments            39 non-null float64
total_payments               125 non-null float64
exercised_stock_options      102 non-null float64
bonus                        82 non-null float64
restricted_stock             110 non-null float64
shared_receipt_with_poi      86 non-null float64
restricted_stock_deferred    18 non-null float64
total_stock_value            126 non-null float64
expenses                     95 non-null float64
loan_advances                4 non-null float64
from_messages                86 non-null float64
other                        93 non-null float64
from_this_person_to_poi      86 non-null float64
director_fees                17 non-null float64
deferred_income              49 n

In [13]:
poi=temp_df['poi']==1
temp_df[poi].count()

poi                          18
salary                       17
to_messages                  14
deferral_payments             5
total_payments               18
exercised_stock_options      12
bonus                        16
restricted_stock             17
shared_receipt_with_poi      14
restricted_stock_deferred     0
total_stock_value            18
expenses                     18
loan_advances                 1
from_messages                14
other                        18
from_this_person_to_poi      14
director_fees                 0
deferred_income              11
long_term_incentive          12
from_poi_to_this_person      14
dtype: int64

In [14]:
temp_df=temp_df.fillna(0)
print "Removing NaN and replacing with 0"
print temp_df.info()

Removing NaN and replacing with 0
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 20 columns):
poi                          146 non-null float64
salary                       146 non-null float64
to_messages                  146 non-null float64
deferral_payments            146 non-null float64
total_payments               146 non-null float64
exercised_stock_options      146 non-null float64
bonus                        146 non-null float64
restricted_stock             146 non-null float64
shared_receipt_with_poi      146 non-null float64
restricted_stock_deferred    146 non-null float64
total_stock_value            146 non-null float64
expenses                     146 non-null float64
loan_advances                146 non-null float64
from_messages                146 non-null float64
other                        146 non-null float64
from_this_person_to_poi      146 non-null float64
director_fees                146 non-null flo

## Outlier detection and data cleaning 

In [15]:
# Determining the outliers for financial datas
financial_features=["salary", "total_payments","long_term_incentive","exercised_stock_options","bonus","restricted_stock", "total_stock_value", "expenses",'deferred_income',"deferral_payments"]
temp_df[financial_features].describe()

,salary,total_payments,long_term_incentive,exercised_stock_options,bonus,restricted_stock,total_stock_value,expenses,deferred_income,deferral_payments
count,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02
mean,3.658114e+05,4.350622e+06,6.646839e+05,4.182736e+06,1.333474e+06,1.749257e+06,5.846018e+06,7.074827e+04,-3.827622e+05,4.387965e+05
std,2.203575e+06,2.693448e+07,4.046072e+06,2.607040e+07,8.094029e+06,1.089995e+07,3.624681e+07,4.327163e+05,2.378250e+06,2.741325e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-4.409300e+04,0.000000e+00,-2.799289e+07,-1.025000e+05
25%,0.000000e+00,9.394475e+04,0.000000e+00,0.000000e+00,0.000000e+00,8.115000e+03,2.288695e+05,0.000000e+00,-3.792600e+04,0.000000e+00
50%,2.105960e+05,9.413595e+05,0.000000e+00,6.082935e+05,3.000000e+05,3.605280e+05,9.659550e+05,2.018200e+04,0.000000e+00,0.000000e+00
75%,2.708505e+05,1.968287e+06,3.750648e+05,1.714221e+06,8.000000e+05,8.145280e+05,2.319991e+06,5.374075e+04,0.000000e+00,9.684500e+03
max,2.670423e+07,3.098866e+08,4.852193e+07,3.117640e+08,9.734362e+07,1.303223e+08,4.345095e+08,5.235198e+06,0.000000e+00,3.208340e+07


In [16]:
temp_df[temp_df["restricted_stock"]<0]

,poi,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,expenses,loan_advances,from_messages,other,from_this_person_to_poi,director_fees,deferred_income,long_term_incentive,from_poi_to_this_person
BHATNAGAR SANJAY,0.0,0.0,523.0,0.0,15456290.0,2604490.0,0.0,-2604490.0,463.0,15456290.0,0.0,0.0,0.0,29.0,137864.0,1.0,137864.0,0.0,0.0,0.0


In [17]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 20 columns):
poi                          146 non-null float64
salary                       146 non-null float64
to_messages                  146 non-null float64
deferral_payments            146 non-null float64
total_payments               146 non-null float64
exercised_stock_options      146 non-null float64
bonus                        146 non-null float64
restricted_stock             146 non-null float64
shared_receipt_with_poi      146 non-null float64
restricted_stock_deferred    146 non-null float64
total_stock_value            146 non-null float64
expenses                     146 non-null float64
loan_advances                146 non-null float64
from_messages                146 non-null float64
other                        146 non-null float64
from_this_person_to_poi      146 non-null float64
director_fees                146 non-null float64
deferred_income              

In [18]:
temp_df[temp_df["salary"]==temp_df["salary"].max()]
# TOTAL seems odd, and it was pointed out in the session that this could be a human error hence this a outlier too.


,poi,salary,to_messages,deferral_payments,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,restricted_stock_deferred,total_stock_value,expenses,loan_advances,from_messages,other,from_this_person_to_poi,director_fees,deferred_income,long_term_incentive,from_poi_to_this_person
TOTAL,0.0,26704229.0,0.0,32083396.0,309886585.0,311764000.0,97343619.0,130322299.0,0.0,-7576788.0,434509511.0,5235198.0,83925000.0,0.0,42667589.0,0.0,1398517.0,-27992891.0,48521928.0,0.0


In [19]:
temp_df=temp_df.drop(index="TOTAL",axis=0)

In [20]:
temp_df[financial_features].loc[(temp_df[financial_features]==0).all(axis=1)].index
# found this one outliers with all finantial features as zero, hence it can be dropped

Index([u'LOCKHART EUGENE E'], dtype='object')

In [21]:
# Dropping Lockhart from data set
temp_df=temp_df.drop(temp_df[financial_features].loc[(temp_df[financial_features]==0).all(axis=1)].index,axis=0)

In [22]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 20 columns):
poi                          144 non-null float64
salary                       144 non-null float64
to_messages                  144 non-null float64
deferral_payments            144 non-null float64
total_payments               144 non-null float64
exercised_stock_options      144 non-null float64
bonus                        144 non-null float64
restricted_stock             144 non-null float64
shared_receipt_with_poi      144 non-null float64
restricted_stock_deferred    144 non-null float64
total_stock_value            144 non-null float64
expenses                     144 non-null float64
loan_advances                144 non-null float64
from_messages                144 non-null float64
other                        144 non-null float64
from_this_person_to_poi      144 non-null float64
director_fees                144 non-null float64
deferred_income              

In [23]:
temp_df.loc['THE TRAVEL AGENCY IN THE PARK'].index

Index([u'poi', u'salary', u'to_messages', u'deferral_payments',
       u'total_payments', u'exercised_stock_options', u'bonus',
       u'restricted_stock', u'shared_receipt_with_poi',
       u'restricted_stock_deferred', u'total_stock_value', u'expenses',
       u'loan_advances', u'from_messages', u'other',
       u'from_this_person_to_poi', u'director_fees', u'deferred_income',
       u'long_term_incentive', u'from_poi_to_this_person'],
      dtype='object')

In [24]:
temp_df=temp_df.drop(['THE TRAVEL AGENCY IN THE PARK'],axis=0)
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 20 columns):
poi                          143 non-null float64
salary                       143 non-null float64
to_messages                  143 non-null float64
deferral_payments            143 non-null float64
total_payments               143 non-null float64
exercised_stock_options      143 non-null float64
bonus                        143 non-null float64
restricted_stock             143 non-null float64
shared_receipt_with_poi      143 non-null float64
restricted_stock_deferred    143 non-null float64
total_stock_value            143 non-null float64
expenses                     143 non-null float64
loan_advances                143 non-null float64
from_messages                143 non-null float64
other                        143 non-null float64
from_this_person_to_poi      143 non-null float64
director_fees                143 non-null float64
deferred_income              

In [26]:
#converting back to dict

data_dict= temp_df.to_dict('index')
features_list=list(temp_df.columns.values)
old_features=list(temp_df.columns.values)
print old_features

['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person']


In [27]:
# Reference module from feature selection: Used for creating new features "fraction_to_poi" and "fraction_from_poi"
def computeFraction( poi_messages, all_messages ):

    if poi_messages!=0 and all_messages!=0:
        fraction = float(poi_messages)/float(all_messages)
        return fraction
    else:
        fraction = 0.0
    



        return fraction

In [28]:
# Used for creating new features "total_profit"
def total_profit(bonus, salary,long_term_incentive):
    profit=float(bonus)+float(salary)+float(long_term_incentive)
    
    return profit
    

In [29]:
submit_dict = {}
for name in data_dict:

    data_point = data_dict[name]

    
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
    
    data_point["fraction_from_poi"] = fraction_from_poi
    
    bonus=data_point["bonus"]
    salary=data_point["salary"]
    long_term_incentive=data_point["long_term_incentive"]


    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
    profit=total_profit(bonus, salary,long_term_incentive)
    submit_dict[name]={"from_poi_to_this_person":fraction_from_poi,
                       "from_this_person_to_poi":fraction_to_poi,
                      "total_profit":total_profit}
    data_point["fraction_to_poi"] = fraction_to_poi
    data_dict[name]["fraction_to_poi"]=fraction_to_poi
    data_dict[name]["fraction_from_poi"]= fraction_from_poi
    data_dict[name]["total_profit"]=profit

In [30]:
#added two additional features in data_dict
total_features=len (data_dict[name_data_point[0]])
print 'After cleaning Total number of features are {}'.format(total_features)

After cleaning Total number of features are 23


In [31]:
print 'After cleaning number of data points {}'.format(len(data_dict))

After cleaning number of data points 143


In [32]:
dict_cols=list(data_dict[name_data_point[0]].keys())

In [33]:
my_dataset = data_dict
for names in dict_cols:
    if names not in features_list:
        
        features_list.append(names)
features_list

['poi',
 'salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'exercised_stock_options',
 'bonus',
 'restricted_stock',
 'shared_receipt_with_poi',
 'restricted_stock_deferred',
 'total_stock_value',
 'expenses',
 'loan_advances',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'director_fees',
 'deferred_income',
 'long_term_incentive',
 'from_poi_to_this_person',
 'fraction_from_poi',
 'total_profit',
 'fraction_to_poi']

In [34]:
print old_features

['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person']


## SVC Classifier with PCA and MinMaX Scalar

In [36]:
print features_list
print old_features

['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person', 'fraction_from_poi', 'total_profit', 'fraction_to_poi']
['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 'exercised_stock_options', 'bonus', 'restricted_stock', 'shared_receipt_with_poi', 'restricted_stock_deferred', 'total_stock_value', 'expenses', 'loan_advances', 'from_messages', 'other', 'from_this_person_to_poi', 'director_fees', 'deferred_income', 'long_term_incentive', 'from_poi_to_this_person']


In [37]:
#With all features
from time import time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split



from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [38]:
# without engineered features
data_old = featureFormat(my_dataset, old_features, sort_keys = True)
labels_old, features_old = targetFeatureSplit(data_old)
features_train_old, features_test_old, labels_train_old, labels_test_old =train_test_split(features_old, labels_old, test_size=0.3, random_state=42)

In [39]:
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)
features_train, features_test, labels_train, labels_test =train_test_split(features, labels, test_size=0.3, random_state=42)

In [40]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.model_selection import StratifiedShuffleSplit
clf_svc=SVC()
pca = RandomizedPCA(n_components=11, whiten=True).fit(features_train)
ss = StratifiedShuffleSplit(n_splits=10, test_size=0.2,random_state = 42)
param_grid_svc={'C': [1e3, 5e3, 1e4, 5e4, 1e5], 'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]}
grid=GridSearchCV(clf_svc,param_grid_svc,cv=ss, scoring = 'f1')

In [41]:
t0=time()
grid.fit(features_train,labels_train)
print "Training time for regular SVC with default parameters is %0.3fs" % (time() - t0)

C:\Users\RATAN\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Training time for regular SVC with default parameters is 2.312s


In [42]:
grid.best_estimator_

SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
y_pred=grid.predict(features_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print "CLASSIFICATION REPORT of SVC "
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX of SVC "
print confusion_matrix(labels_test, y_pred, labels=[0,1])
print "--------The f1-score of SVC classifer------: "
print f1_score(labels_test,y_pred)

CLASSIFICATION REPORT of SVC 
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94        38
         1.0       0.00      0.00      0.00         5

   micro avg       0.88      0.88      0.88        43
   macro avg       0.44      0.50      0.47        43
weighted avg       0.78      0.88      0.83        43

CONFUSION MATRIX of SVC 
[[38  0]
 [ 5  0]]
--------The f1-score of SVC classifer------: 
0.0


C:\Users\RATAN\Anaconda2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [44]:
#Regular SVC without default parameters

scaler = MinMaxScaler().fit(features_train)

#Minmax transform
X_train_scaler =  scaler.transform(features_train)
X_test_scaler =scaler.transform(features_test)
### PCA transform
X_train_pca = pca.transform(features_train)
X_test_pca = pca.transform(features_test)
## PCA and scaler
X_train_pca_scaler=pca.transform(X_train_scaler)
X_test_pca_scaler=pca.transform(X_test_scaler)


In [45]:
t0=time()
grid.fit(X_train_pca,labels_train)
print "Training time for regular SVC with PCA is %0.3fs" % (time() - t0)

Training time for regular SVC with PCA is 2.377s


In [46]:
grid.best_estimator_

SVC(C=10000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
y_pred=grid.predict(X_test_pca)
print "CLASSIFICATION REPORT of SVC with PCA "
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX of SVC with PCA"
print confusion_matrix(labels_test, y_pred, labels=[0,1])
print "--------The f1-score of SVC classifer with PCA------: "
print f1_score(labels_test,y_pred)

CLASSIFICATION REPORT of SVC with PCA 
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91        38
         1.0       0.33      0.40      0.36         5

   micro avg       0.84      0.84      0.84        43
   macro avg       0.63      0.65      0.64        43
weighted avg       0.85      0.84      0.84        43

CONFUSION MATRIX of SVC with PCA
[[34  4]
 [ 3  2]]
--------The f1-score of SVC classifer with PCA------: 
0.3636363636363636


In [48]:
t0=time()
grid.fit(X_train_scaler,labels_train)
print "Training time for regular SVC with scaler is %0.3fs" % (time() - t0)

Training time for regular SVC with scaler is 1.966s


In [49]:
y_pred=grid.predict(X_test_scaler)
print "CLASSIFICATION REPORT of SVC with scaler  "
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX of SVC with with scaler"
print confusion_matrix(labels_test, y_pred, labels=[0,1])
print "--------The f1-score of SVC with scaler------: "
print f1_score(labels_test,y_pred)

CLASSIFICATION REPORT of SVC with scaler  
              precision    recall  f1-score   support

         0.0       0.90      0.95      0.92        38
         1.0       0.33      0.20      0.25         5

   micro avg       0.86      0.86      0.86        43
   macro avg       0.62      0.57      0.59        43
weighted avg       0.83      0.86      0.84        43

CONFUSION MATRIX of SVC with with scaler
[[36  2]
 [ 4  1]]
--------The f1-score of SVC with scaler------: 
0.25


## Naive Bayes Gaussian

In [50]:
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest, f_classif
clf_gnb=GaussianNB()
#Using SKbest module to choose best number of features for classification
skb=SelectKBest()

In [51]:
from sklearn.pipeline import Pipeline
pipe_2=Pipeline([('SKB',skb),('GNB', clf_gnb)])

pipe_2.get_params().keys()

['GNB__priors',
 'memory',
 'SKB__k',
 'GNB',
 'SKB__score_func',
 'steps',
 'GNB__var_smoothing',
 'SKB']

In [52]:
param_grid={"SKB__k":[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]}
clf_grid_gnb=GridSearchCV(pipe_2,param_grid,cv=ss, scoring = 'f1')

In [53]:
t0=time()
clf_grid_gnb.fit(features,labels)
print "training done for GNB in %0.3fs" % (time() - t0)

training done for GNB in 1.647s


In [54]:
features_list

['poi',
 'salary',
 'to_messages',
 'deferral_payments',
 'total_payments',
 'exercised_stock_options',
 'bonus',
 'restricted_stock',
 'shared_receipt_with_poi',
 'restricted_stock_deferred',
 'total_stock_value',
 'expenses',
 'loan_advances',
 'from_messages',
 'other',
 'from_this_person_to_poi',
 'director_fees',
 'deferred_income',
 'long_term_incentive',
 'from_poi_to_this_person',
 'fraction_from_poi',
 'total_profit',
 'fraction_to_poi']

In [55]:
clf_grid_gnb.best_estimator_

Pipeline(memory=None,
     steps=[('SKB', SelectKBest(k=6, score_func=<function f_classif at 0x000000000E2B2748>)), ('GNB', GaussianNB(priors=None, var_smoothing=1e-09))])

In [56]:
clf=clf_grid_gnb.best_estimator_
clf.fit(features_train,labels_train)
y_pred=clf.predict(features_test)

In [57]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print "CLASSIFICATION REPORT for Gaussian"
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX for Gaussian"
print(pd.DataFrame(confusion_matrix(labels_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))

CLASSIFICATION REPORT for Gaussian
              precision    recall  f1-score   support

         0.0       0.95      0.92      0.93        38
         1.0       0.50      0.60      0.55         5

   micro avg       0.88      0.88      0.88        43
   macro avg       0.72      0.76      0.74        43
weighted avg       0.89      0.88      0.89        43

CONFUSION MATRIX for Gaussian
     pred_neg  pred_pos
neg        35         3
pos         2         3


In [58]:
print "---------Accuracy of GNB------------"
print accuracy_score(labels_test, y_pred)
print "\n"
print "-------------Precision of GNB ----------- "
print precision_score(labels_test,y_pred)
print "\n"
print "----------Recall of  GNB -----------    "
print recall_score(labels_test,y_pred)
print "\n"
print "--------The f1-score of GNB------: "
print f1_score(labels_test,y_pred)

---------Accuracy of GNB------------
0.8837209302325582


-------------Precision of GNB ----------- 
0.5


----------Recall of  GNB -----------    
0.6


--------The f1-score of GNB------: 
0.5454545454545454


## Adaboost

In [59]:
from sklearn.ensemble import AdaBoostClassifier
clf_ada = AdaBoostClassifier(random_state=0)
pipe_ada=Pipeline([('SKB',skb),('ADA', clf_ada)])
pipe_ada.get_params().keys()

['memory',
 'ADA__n_estimators',
 'ADA__base_estimator',
 'SKB',
 'ADA__algorithm',
 'SKB__k',
 'SKB__score_func',
 'ADA__learning_rate',
 'steps',
 'ADA',
 'ADA__random_state']

In [60]:
param_grid={"SKB__k":[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],'ADA__algorithm':['SAMME','SAMME.R'] }
clf_grid_ada=GridSearchCV(pipe_ada,param_grid,cv=ss, scoring = 'f1')

clf_grid_ada.fit(features,labels)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('SKB', SelectKBest(k=10, score_func=<function f_classif at 0x000000000E2B2748>)), ('ADA', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=0))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'ADA__algorithm': ['SAMME', 'SAMME.R'], 'SKB__k': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [61]:
clf_ada=clf_grid_ada.best_estimator_

clf_ada.fit(features_train,labels_train)
y_pred=clf_ada.predict(features_test)

In [62]:
print "CLASSIFICATION REPORT for Trees"
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX for Trees"
print(pd.DataFrame(confusion_matrix(labels_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))

CLASSIFICATION REPORT for Trees
              precision    recall  f1-score   support

         0.0       0.92      0.89      0.91        38
         1.0       0.33      0.40      0.36         5

   micro avg       0.84      0.84      0.84        43
   macro avg       0.63      0.65      0.64        43
weighted avg       0.85      0.84      0.84        43

CONFUSION MATRIX for Trees
     pred_neg  pred_pos
neg        34         4
pos         3         2


In [63]:
print "---------Accuracy of ADABOOST------------"
print accuracy_score(labels_test, y_pred)
print "\n"
print "-------------Precision of ADABOOST ----------- "
print precision_score(labels_test,y_pred)
print "\n"
print "----------Recall of  ADABOOST -----------    "
print recall_score(labels_test,y_pred)
print "\n"
print "--------The f1-score of ADABOOST------: "
print f1_score(labels_test,y_pred)

---------Accuracy of ADABOOST------------
0.8372093023255814


-------------Precision of ADABOOST ----------- 
0.3333333333333333


----------Recall of  ADABOOST -----------    
0.4


--------The f1-score of ADABOOST------: 
0.3636363636363636


## Decision Trees

In [64]:
from sklearn.tree import DecisionTreeClassifier
clf_tree=DecisionTreeClassifier(random_state=0)
t0=time()
clf_tree.fit(features_train,labels_train)
print "DT training done for regular tree in %0.3fs" % (time() - t0)

DT training done for regular tree in 0.003s


In [65]:
y_pred=clf_tree.predict(features_test)

In [66]:
print "CLASSIFICATION REPORT"
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX"
print(pd.DataFrame(confusion_matrix(labels_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))

CLASSIFICATION REPORT
              precision    recall  f1-score   support

         0.0       0.89      0.89      0.89        38
         1.0       0.20      0.20      0.20         5

   micro avg       0.81      0.81      0.81        43
   macro avg       0.55      0.55      0.55        43
weighted avg       0.81      0.81      0.81        43

CONFUSION MATRIX
     pred_neg  pred_pos
neg        34         4
pos         4         1


In [67]:
print "---------Accuracy of Decision Tree Default------------"
print accuracy_score(labels_test, y_pred)
print "\n"
print "-------------Precision of Decision Tree Default ----------- "
print precision_score(labels_test,y_pred)
print "\n"
print "----------Recall of  Decision Tree Default -----------    "
print recall_score(labels_test,y_pred)
print "\n"
print "--------The f1-score of Decision Tree Default------: "
print f1_score(labels_test,y_pred)

---------Accuracy of Decision Tree Default------------
0.813953488372093


-------------Precision of Decision Tree Default ----------- 
0.2


----------Recall of  Decision Tree Default -----------    
0.2


--------The f1-score of Decision Tree Default------: 
0.20000000000000004


In [68]:
pipe_tree=Pipeline([('SKB',skb),('tree', clf_tree)])
pipe_tree.get_params().keys()

['tree__criterion',
 'tree__min_weight_fraction_leaf',
 'tree__splitter',
 'tree__class_weight',
 'tree__min_samples_split',
 'tree__min_impurity_split',
 'memory',
 'tree',
 'SKB__k',
 'tree__min_impurity_decrease',
 'tree__random_state',
 'SKB__score_func',
 'steps',
 'tree__max_leaf_nodes',
 'tree__max_depth',
 'SKB',
 'tree__presort',
 'tree__max_features',
 'tree__min_samples_leaf']

In [69]:
param_grid={"SKB__k":[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],'tree__criterion':['entropy'],'tree__min_samples_split':[2,5,7,10,12,15],'tree__max_depth':[1,2,3,4,5]}
clf_grid_tree=GridSearchCV(pipe_tree,param_grid,cv=ss, scoring = 'f1')

In [70]:
t0=time()
clf_grid_tree.fit(features,labels)
print "training done for Tree in pipeline in %0.3fs" % (time() - t0)

training done for Tree in pipeline in 59.208s


In [71]:
clf_grid_tree.best_estimator_

Pipeline(memory=None,
     steps=[('SKB', SelectKBest(k=19, score_func=<function f_classif at 0x000000000E2B2748>)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [72]:
clf_tree=clf_grid_tree.best_estimator_
clf_tree.fit(features_train,labels_train)
y_pred=clf_tree.predict(features_test)

In [73]:
print "CLASSIFICATION REPORT for Trees With New Features"
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX for Trees With New Features"
print(pd.DataFrame(confusion_matrix(labels_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))

CLASSIFICATION REPORT for Trees
              precision    recall  f1-score   support

         0.0       0.97      0.95      0.96        38
         1.0       0.67      0.80      0.73         5

   micro avg       0.93      0.93      0.93        43
   macro avg       0.82      0.87      0.84        43
weighted avg       0.94      0.93      0.93        43

CONFUSION MATRIX for Trees
     pred_neg  pred_pos
neg        36         2
pos         1         4


In [74]:
print "---------Accuracy of Decision Tree With New Features------------"
print accuracy_score(labels_test, y_pred)
print "\n"
print "-------------Precision of Decision Tree With New Features ----------- "
print precision_score(labels_test,y_pred)
print "\n"
print "----------Recall of  Decision Tree With New Features -----------    "
print recall_score(labels_test,y_pred)
print "\n"
print "--------The f1-score of Decision Tree With New Features------: "
print f1_score(labels_test,y_pred)

---------Accuracy of Decision Tree------------
0.9302325581395349


-------------Precision of Decision Tree ----------- 
0.6666666666666666


----------Recall of  Decision Tree -----------    
0.8


--------The f1-score of Decision Tree------: 
0.7272727272727272


In [76]:
# Impact of Engineered features
param_grid_old={"SKB__k":[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],'tree__criterion':['entropy'],'tree__min_samples_split':[2,5,7,10,12,15],'tree__max_depth':[1,2,3,4,5]}
clf_grid_tree=GridSearchCV(pipe_tree,param_grid_old,cv=ss, scoring = 'f1')
t0=time()
clf_grid_tree.fit(features_old,labels_old)
print "training done for Tree in pipeline without new features in %0.3fs" % (time() - t0)

training done for Tree in pipeline without new features in 55.883s


In [77]:
clf_grid_tree.best_estimator_

Pipeline(memory=None,
     steps=[('SKB', SelectKBest(k=13, score_func=<function f_classif at 0x000000000E2B2748>)), ('tree', DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [78]:
clf_tree_old=clf_grid_tree.best_estimator_
clf_tree_old.fit(features_train_old,labels_train_old)
y_pred=clf_tree_old.predict(features_test_old)

In [79]:
print "CLASSIFICATION REPORT for Trees without New features"
print classification_report(labels_test, y_pred)
print "CONFUSION MATRIX for Trees without New features"
print(pd.DataFrame(confusion_matrix(labels_test, y_pred),
                 columns=['pred_neg', 'pred_pos'], index=['neg', 'pos']))

CLASSIFICATION REPORT for Trees without New features
              precision    recall  f1-score   support

         0.0       0.94      0.79      0.86        38
         1.0       0.27      0.60      0.37         5

   micro avg       0.77      0.77      0.77        43
   macro avg       0.61      0.69      0.62        43
weighted avg       0.86      0.77      0.80        43

CONFUSION MATRIX for Trees without New features
     pred_neg  pred_pos
neg        30         8
pos         2         3


In [80]:
print "---------Accuracy of Decision Tree without new features------------"
print accuracy_score(labels_test, y_pred)
print "\n"
print "-------------Precision of Decision Tree without new features ----------- "
print precision_score(labels_test,y_pred)
print "\n"
print "----------Recall of  Decision Tree without new features -----------    "
print recall_score(labels_test,y_pred)
print "\n"
print "--------The f1-score of Decision Tree without new features------: "
print f1_score(labels_test,y_pred)

---------Accuracy of Decision Tree without new features------------
0.7674418604651163


-------------Precision of Decision Tree without new features ----------- 
0.2727272727272727


----------Recall of  Decision Tree without new features -----------    
0.6


--------The f1-score of Decision Tree without new features------: 
0.37499999999999994


In [177]:
dump_classifier_and_data(clf_tree, my_dataset, features_list)